In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from transition_util import *
import multiprocessing as mp
import pickle
import math
import os

In [3]:
flight_data = pd.read_csv("../../samples_with_jfk_landing_flag.csv")
flight_data = flight_data.sort_values(by = ['id', 'ts']).reset_index(drop = True)
# flight_data['jfk_landing_flag'] = (flight_data['jfk_landing_flag'] & (flight_data['destination'] == 'JFK'))
flight_data['lon'] = flight_data['lon'].clip(-78, -68)
flight_data['lat'] = flight_data['lat'].clip(35, 45)
flight_data['altitude'] = flight_data['altitude'].clip(0, 50000)
conversion_x = 9.26/((flight_data['lat'] * math.pi / 180).apply(math.cos) * 111.699)
conversion_y = 9.26/111.699
conversion_z = 500
unit_time = 10
flight_data['x'] = ((flight_data['lon'] + 78)/conversion_x).apply(np.ceil).apply(int)
flight_data['y'] = ((flight_data['lat'] - 35)/conversion_y).apply(np.ceil).apply(int)
flight_data['z'] = ((flight_data['altitude']) / conversion_z).apply(np.ceil).apply(int)

In [ ]:
start_ts_df = flight_data[['id', 'ts']].groupby(['id']).first().reset_index(drop = False)
start_ts_df.columns = ['id', 'start_ts']
flight_data = flight_data.merge(start_ts_df)
flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)
flight_data['t'] = (flight_data['ts'] - flight_data['start_ts'])/unit_time
flight_data.head()

In [ ]:
flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)

In [6]:
if not(os.path.isfile("transitions1_xyzt.pkl")):
    transitions1_start_xyzt = {}
    condition = True
    previous_id = ''
    flight_data1 = flight_data
    flight_data1['t'] = ((flight_data1['ts'] - flight_data1['start_ts'])/unit_time).apply(np.ceil).apply(int)
    flight_data1['last_ts'] = flight_data1[['id', 'ts']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_t'] = flight_data1[['id', 't']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_x'] = flight_data1[['id', 'x']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_y'] = flight_data1[['id', 'y']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_z'] = flight_data1[['id', 'z']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_lon'] = flight_data1[['id', 'lon']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_lat'] = flight_data1[['id', 'lat']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_climb_rate'] = flight_data1[['id', 'climb_rate']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1['last_altitude'] = flight_data1[['id', 'altitude']].set_index("id").groupby(level = "id").shift(1).reset_index(drop = True)
    flight_data1 = flight_data1[~flight_data1['last_ts'].apply(np.isnan)]
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data1 = flight_data1[(flight_data1['last_x'] != flight_data1['x']) |
                              (flight_data1['last_y'] != flight_data1['y']) |
                              (flight_data1['last_z'] != flight_data1['z']) |
                              (flight_data1['last_t'] != flight_data1['t'])]
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data1['last_x'] = flight_data1['last_x'].apply(int)
    flight_data1['last_y'] = flight_data1['last_y'].apply(int)
    flight_data1['last_z'] = flight_data1['last_z'].apply(int)
    flight_data1['last_t'] = flight_data1['last_t'].apply(int)
    
    start_df = flight_data1[['id', 'last_ts', 'last_lon', 'last_lat', 'last_altitude']].groupby(['id']).first().reset_index(drop = False)
    start_df.columns = ['id', 'start_ts', 'start_lon', 'start_lat', 'start_altitude']
    flight_data1 = flight_data1.merge(start_df)
    flight_data1 = flight_data1.sort_values(['id', 'ts']).reset_index(drop = True)
    
    # Retaining only aircraft IDs with destination == "JFK" in all rows
    df1 = flight_data[flight_data['jfk_landing_flag']]
    df1 = df1.sort_values(['id', 'ts']).reset_index(drop = True)
    df1 = df1.groupby(['id']).apply(lambda x: (x['destination'] == "JFK").mean()).reset_index(drop = False)
    df1 = df1[['id']][df1[0] == 1.0]
    landing_ac_data = df1.merge(flight_data1)
    landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
    
    # Retaining only IDs that have successfully landed at JFK (small range)
    last_df = landing_ac_data.groupby(['id']).last().reset_index(drop = False)
    last_df['coords'] = last_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
    last_df = last_df[last_df['coords'].apply(lambda x: x[2] <= 3)]
    print(last_df['coords'].value_counts())
    landing_ac_data = landing_ac_data.merge(last_df[['id']])
    landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
    landed_xyzt = set(last_df['coords'])
    print(landed_xyz)
    
    for i in range(landing_ac_data.shape[0]):
        id1 = landing_ac_data['id'].iloc[i]
        last_x = landing_ac_data['last_x'].iloc[i]
        last_y = landing_ac_data['last_y'].iloc[i]
        last_z = landing_ac_data['last_z'].iloc[i]
        last_t = landing_ac_data['last_t'].iloc[i]
        x = landing_ac_data['x'].iloc[i]
        y = landing_ac_data['y'].iloc[i]
        z = landing_ac_data['z'].iloc[i]
        t = landing_ac_data['t'].iloc[i]
        condition = (previous_id != id1)
        if (not((last_x, last_y, last_z) in landed_xyz) and condition):
            last_row = True
            try:
                transitions1_start_xyzt[(last_x, last_y, last_z, last_t)][(x, y, z, t)] += 1
            except:
                try:
                    transitions1_start_xyzt[(last_x, last_y, last_z, last_t)][(x, y, z, t)] = 1
                except:
                    transitions1_start_xyzt[(last_x, last_y, last_z, last_t)] = {(x, y, z, t): 1}
        elif last_row:
            previous_id = id1
            last_row = False
            try:
                transitions1_start_xyzt[(last_x, last_y, last_z, last_t)][(x, y, z, t)] += 1
            except:
                try:
                    transitions1_start_xyzt[(last_x, last_y, last_z, last_t)][(x, y, z, t)] = 1
                except:
                    transitions1_start_xyzt[(last_x, last_y, last_z, last_t)] = {(x, y, z, t): 1}
#         else:
#             print(previous_id)
    
    pickle.dump(transitions1_start_xyzt, open("transitions1_start_xyzt.pkl", "wb"))
    pickle.dump(landed_xyzt, open("landed_start_xyzt.pkl", "wb"))
else:
    transitions1_start_xyzt = pickle.load(open("transitions1_start_xyzt.pkl", "rb"))
    landed_xyzt = pickle.load(open("landed_start_xyzt.pkl", "rb"))

(39, 68, 0)    3869
(39, 69, 0)    2928
(40, 68, 0)       2
(40, 69, 0)       1
Name: coords, dtype: int64
{(39, 68, 0), (39, 69, 0), (40, 69, 0), (40, 68, 0)}


NameError: name 'landed_xyzt' is not defined

In [11]:
lis1 = [transitions1_start_xyzt[key].values() for key in transitions1_start_xyzt.keys()]
lis2 = unlist(lis1)
lis2 = pd.Series(lis2)
print(lis2.value_counts())
print(len(lis2))

1      930894
2      127328
3       46849
4       23776
5       14077
6        9121
7        6418
8        4669
9        3521
10       2664
11       2179
12       1697
13       1317
14       1165
15        926
16        819
17        685
18        616
19        536
20        401
21        377
22        334
23        297
24        290
25        264
26        214
27        214
28        155
29        152
30        140
        ...  
187         1
188         1
191         1
212         1
192         1
168         1
219         1
216         1
203         1
207         1
263         1
115         1
156         1
151         1
87          1
93          1
101         1
102         1
103         1
106         1
109         1
112         1
126         1
129         1
134         1
135         1
136         1
142         1
146         1
268         1
Length: 141, dtype: int64
1184170
